## 30s k-fold

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, Dropout
from keras.layers import Activation, BatchNormalization
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from keras.callbacks import TensorBoard, EarlyStopping
np.set_printoptions(suppress=True)

trainD = np.load("/home/hsiehch/30s/train_data.npy")
trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationD = np.load("/home/hsiehch/30s/validation_data.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")
testD = np.load("/home/hsiehch/30s/test_data.npy")
testL = np.load("/home/hsiehch/30s/test_label.npy")

trainD = np.append(trainD, validationD, axis=0)
trainL = np.append(trainL, validationL, axis=0)
trainD = np.append(trainD, testD, axis=0)
trainL = np.append(trainL, testL, axis=0)

trainData = trainD.reshape((trainD.shape[0], trainD.shape[1], 1))
trainLabel = np_utils.to_categorical(trainL, 4)
print('Train Data:', trainData.shape)
print('Train Label: ', trainLabel.shape)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train Data: (10151, 9000, 1)
Train Label:  (10151, 4)


In [2]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
print(kf)

training_data = []
training_label = []
validation_cate_label = []
validation_data = []
validation_label = []

for train_index, test_index in kf.split(trainData):
    print('trian:', train_index, 'len', len(train_index), 'test:', test_index, 'len', len(test_index))
    training_data.append(trainData[train_index])
    training_label.append(trainLabel[train_index])
    validation_data.append(trainData[test_index])
    validation_label.append(trainLabel[test_index])
    validation_cate_label.append(trainL[test_index])

training_data = np.array(training_data)
training_label = np.array(training_label)
validation_data = np.array(validation_data)
validation_label = np.array(validation_label)
validation_cate_label = np.array(validation_cate_label)


KFold(n_splits=5, random_state=None, shuffle=True)
trian: [    2     5     6 ... 10148 10149 10150] len 8120 test: [    0     1     3 ... 10128 10135 10146] len 2031
trian: [    0     1     3 ... 10148 10149 10150] len 8121 test: [    2    11    13 ... 10140 10141 10143] len 2030
trian: [    0     1     2 ... 10146 10148 10150] len 8121 test: [    5    14    17 ... 10145 10147 10149] len 2030
trian: [    0     1     2 ... 10146 10147 10149] len 8121 test: [    7    10    20 ... 10139 10148 10150] len 2030
trian: [    0     1     2 ... 10148 10149 10150] len 8121 test: [    6     9    31 ... 10134 10138 10142] len 2030


In [3]:
training_data[0].shape

(8120, 9000, 1)

In [4]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import itertools

def create_model():
    model = Sequential() 
    model.add(Conv1D(filters = 32, kernel_size = 7, input_shape = (trainData.shape[1], 1)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 32, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(4, activation = "softmax"))

    return model

def calculate_avg_F1(scores):
    tmp = [0,0,0,0]
    for i in scores:
        tmp += i
    tmp /= 5
    return tmp

def run_model(batch_S, learning_rate=0.001):
    
    Scores = []
    for X_train, Y_train, X_val, Y_val, val_cat in zip(training_data, training_label, validation_data, validation_label, validation_cate_label):

        model = create_model()
        adam = Adam(lr = learning_rate)
        model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])
        early_stop = EarlyStopping(patience=20)
        train_history_1 = model.fit(x = X_train, 
                                    y = Y_train,
                                    epochs=80,
                                    validation_data=(X_val, Y_val),
                                    callbacks=[early_stop],
                                    batch_size=batch_S, 
                                    verbose=1)
        evaluation = model.evaluate(x = X_val, y = Y_val)
        print('Loss: {:.3f}, Accuracy: {:.3f}'.format(evaluation[0], evaluation[1]))

        validation_prediction = model.predict_classes(X_val, batch_size=batch_S)
        score = f1_score(val_cat, validation_prediction, average=None)
        Scores.append(score)
        del model
    
    result = calculate_avg_F1(Scores)
    F1_result.append(result)
    del Scores
    

        
F1_result = []
hyperparameter_index = []
batch_S = [70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]

for bat in batch_S:
    hyperparameter_index.append([bat])
    run_model(bat)

print('Finish training!')

Train on 8120 samples, validate on 2031 samples
Epoch 1/80
8120/8120 [==============================] - 14s 2ms/step - loss: 1.0419 - acc: 0.5698 - val_loss: 0.9277 - val_acc: 0.6032
Epoch 2/80
8120/8120 [==============================] - 7s 912us/step - loss: 0.8911 - acc: 0.5924 - val_loss: 0.8599 - val_acc: 0.6460
Epoch 3/80
8120/8120 [==============================] - 8s 955us/step - loss: 0.7992 - acc: 0.6499 - val_loss: 0.7201 - val_acc: 0.6982
Epoch 4/80
8120/8120 [==============================] - 8s 950us/step - loss: 0.6983 - acc: 0.7028 - val_loss: 0.7094 - val_acc: 0.7420
Epoch 5/80
8120/8120 [==============================] - 8s 959us/step - loss: 0.6504 - acc: 0.7361 - val_loss: 0.6336 - val_acc: 0.7627
Epoch 6/80
8120/8120 [==============================] - 8s 959us/step - loss: 0.5895 - acc: 0.7676 - val_loss: 0.6580 - val_acc: 0.7696
Epoch 7/80
8120/8120 [==============================] - 8s 947us/step - loss: 0.5756 - acc: 0.7754 - val_loss: 0.5368 - val_acc: 0.8016
E

8121/8121 [==============================] - 8s 965us/step - loss: 0.5830 - acc: 0.7681 - val_loss: 0.4954 - val_acc: 0.8084
Epoch 11/80
8121/8121 [==============================] - 8s 975us/step - loss: 0.5373 - acc: 0.7884 - val_loss: 0.4870 - val_acc: 0.8084
Epoch 12/80
8121/8121 [==============================] - 8s 972us/step - loss: 0.5246 - acc: 0.8009 - val_loss: 0.4456 - val_acc: 0.8315
Epoch 13/80
8121/8121 [==============================] - 8s 979us/step - loss: 0.5134 - acc: 0.8080 - val_loss: 0.5481 - val_acc: 0.7911
Epoch 14/80
8121/8121 [==============================] - 8s 971us/step - loss: 0.5781 - acc: 0.7657 - val_loss: 0.4458 - val_acc: 0.8330
Epoch 15/80
8121/8121 [==============================] - 8s 966us/step - loss: 0.4978 - acc: 0.8132 - val_loss: 0.4537 - val_acc: 0.8261
Epoch 16/80
8121/8121 [==============================] - 8s 952us/step - loss: 0.4728 - acc: 0.8227 - val_loss: 0.4513 - val_acc: 0.8345
Epoch 17/80
8121/8121 [==============================

Epoch 8/80
8121/8121 [==============================] - 8s 964us/step - loss: 0.5624 - acc: 0.7825 - val_loss: 0.5310 - val_acc: 0.7980
Epoch 9/80
8121/8121 [==============================] - 8s 963us/step - loss: 0.5386 - acc: 0.7990 - val_loss: 0.5228 - val_acc: 0.8039
Epoch 10/80
8121/8121 [==============================] - 8s 969us/step - loss: 0.5163 - acc: 0.8041 - val_loss: 0.5354 - val_acc: 0.8148
Epoch 11/80
8121/8121 [==============================] - 8s 933us/step - loss: 0.4970 - acc: 0.8164 - val_loss: 0.5042 - val_acc: 0.8128
Epoch 12/80
8121/8121 [==============================] - 8s 973us/step - loss: 0.4889 - acc: 0.8211 - val_loss: 0.4922 - val_acc: 0.8246
Epoch 13/80
8121/8121 [==============================] - 8s 962us/step - loss: 0.4796 - acc: 0.8256 - val_loss: 0.4966 - val_acc: 0.8197
Epoch 14/80
8121/8121 [==============================] - 8s 936us/step - loss: 0.4630 - acc: 0.8317 - val_loss: 0.4709 - val_acc: 0.8365
Epoch 15/80
8121/8121 [====================

Epoch 16/80
8121/8121 [==============================] - 8s 936us/step - loss: 0.4938 - acc: 0.8158 - val_loss: 0.4597 - val_acc: 0.8300
Epoch 17/80
8121/8121 [==============================] - 8s 970us/step - loss: 0.4461 - acc: 0.8346 - val_loss: 0.4849 - val_acc: 0.8138
Epoch 18/80
8121/8121 [==============================] - 8s 929us/step - loss: 0.4485 - acc: 0.8366 - val_loss: 0.4624 - val_acc: 0.8266
Epoch 19/80
8121/8121 [==============================] - 8s 968us/step - loss: 0.4307 - acc: 0.8414 - val_loss: 0.4823 - val_acc: 0.8251
Epoch 20/80
8121/8121 [==============================] - 8s 933us/step - loss: 0.4280 - acc: 0.8446 - val_loss: 0.4707 - val_acc: 0.8355
Epoch 21/80
8121/8121 [==============================] - 8s 962us/step - loss: 0.4202 - acc: 0.8478 - val_loss: 0.5468 - val_acc: 0.8172
Epoch 22/80
8121/8121 [==============================] - 8s 939us/step - loss: 0.4776 - acc: 0.8189 - val_loss: 0.5942 - val_acc: 0.7828
Epoch 23/80
8121/8121 [==================

Epoch 76/80
2030/2030 [==============================] - 1s 326us/step
Loss: 0.466, Accuracy: 0.845
Train on 8121 samples, validate on 2030 samples
Epoch 1/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.9990 - acc: 0.5780 - val_loss: 0.9022 - val_acc: 0.5872
Epoch 2/80
8121/8121 [==============================] - 8s 948us/step - loss: 0.8909 - acc: 0.6145 - val_loss: 0.7973 - val_acc: 0.6576
Epoch 3/80
8121/8121 [==============================] - 8s 954us/step - loss: 0.7889 - acc: 0.6645 - val_loss: 0.6857 - val_acc: 0.7059
Epoch 4/80
8121/8121 [==============================] - 8s 928us/step - loss: 0.7135 - acc: 0.7030 - val_loss: 0.6501 - val_acc: 0.7355
Epoch 5/80
8121/8121 [==============================] - 8s 931us/step - loss: 0.6229 - acc: 0.7537 - val_loss: 0.7545 - val_acc: 0.6966
Epoch 6/80
8121/8121 [==============================] - 8s 931us/step - loss: 0.6003 - acc: 0.7639 - val_loss: 0.5499 - val_acc: 0.7857
Epoch 7/80
8121/8121 [===============

...

8121/8121 [==============================] - 8s 1ms/step - loss: 0.3702 - acc: 0.8654 - val_loss: 0.4848 - val_acc: 0.8394
Epoch 33/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3624 - acc: 0.8710 - val_loss: 0.4475 - val_acc: 0.8522
Epoch 34/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3559 - acc: 0.8724 - val_loss: 0.4501 - val_acc: 0.8502
Epoch 35/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3569 - acc: 0.8717 - val_loss: 0.4434 - val_acc: 0.8438
Epoch 36/80
8121/8121 [==============================] - 8s 995us/step - loss: 0.3447 - acc: 0.8742 - val_loss: 0.4963 - val_acc: 0.8315
Epoch 37/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3469 - acc: 0.8698 - val_loss: 0.5029 - val_acc: 0.8315
Epoch 38/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3391 - acc: 0.8754 - val_loss: 0.4411 - val_acc: 0.8527
Epoch 39/80
2030/2030 [==============================] - 1s 469us

8121/8121 [==============================] - 8s 977us/step - loss: 0.7994 - acc: 0.6502 - val_loss: 0.7589 - val_acc: 0.6419
Epoch 4/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.7207 - acc: 0.6908 - val_loss: 0.6627 - val_acc: 0.7187
Epoch 5/80
8121/8121 [==============================] - 8s 975us/step - loss: 0.6671 - acc: 0.7264 - val_loss: 0.6429 - val_acc: 0.7601
Epoch 6/80
8121/8121 [==============================] - 8s 977us/step - loss: 0.6202 - acc: 0.7486 - val_loss: 0.5527 - val_acc: 0.7680
Epoch 7/80
8121/8121 [==============================] - 8s 974us/step - loss: 0.5983 - acc: 0.7653 - val_loss: 0.5653 - val_acc: 0.7813
Epoch 8/80
8121/8121 [==============================] - 8s 999us/step - loss: 0.5820 - acc: 0.7700 - val_loss: 0.5034 - val_acc: 0.8192
Epoch 9/80
8121/8121 [==============================] - 8s 994us/step - loss: 0.5512 - acc: 0.7882 - val_loss: 0.5049 - val_acc: 0.8030
Epoch 10/80
8121/8121 [==============================] - 8s 9

8121/8121 [==============================] - 8s 978us/step - loss: 0.4539 - acc: 0.8324 - val_loss: 0.4854 - val_acc: 0.8158
Epoch 16/80
8121/8121 [==============================] - 8s 972us/step - loss: 0.4487 - acc: 0.8324 - val_loss: 0.4624 - val_acc: 0.8340
Epoch 17/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.4360 - acc: 0.8402 - val_loss: 0.4607 - val_acc: 0.8172
Epoch 18/80
8121/8121 [==============================] - 8s 989us/step - loss: 0.4401 - acc: 0.8402 - val_loss: 0.4713 - val_acc: 0.8300
Epoch 19/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.4246 - acc: 0.8444 - val_loss: 0.4866 - val_acc: 0.8222
Epoch 20/80
8121/8121 [==============================] - 8s 999us/step - loss: 0.4208 - acc: 0.8473 - val_loss: 0.4981 - val_acc: 0.8271
Epoch 21/80
8121/8121 [==============================] - 8s 985us/step - loss: 0.4236 - acc: 0.8412 - val_loss: 0.4535 - val_acc: 0.8374
Epoch 22/80
8121/8121 [==============================] - 

8121/8121 [==============================] - 8s 1ms/step - loss: 0.3792 - acc: 0.8590 - val_loss: 0.4402 - val_acc: 0.8502
Epoch 27/80
8121/8121 [==============================] - 8s 977us/step - loss: 0.3630 - acc: 0.8663 - val_loss: 0.4509 - val_acc: 0.8389
Epoch 28/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3711 - acc: 0.8617 - val_loss: 0.4518 - val_acc: 0.8429
Epoch 29/80
8121/8121 [==============================] - 8s 997us/step - loss: 0.3630 - acc: 0.8718 - val_loss: 0.4402 - val_acc: 0.8468
Epoch 30/80
8121/8121 [==============================] - 8s 975us/step - loss: 0.3499 - acc: 0.8762 - val_loss: 0.4651 - val_acc: 0.8305
Epoch 31/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3469 - acc: 0.8686 - val_loss: 0.4498 - val_acc: 0.8443
Epoch 32/80
8121/8121 [==============================] - 8s 994us/step - loss: 0.3561 - acc: 0.8660 - val_loss: 0.4652 - val_acc: 0.8389
Epoch 33/80
8121/8121 [==============================] - 8s

8121/8121 [==============================] - 8s 983us/step - loss: 0.4047 - acc: 0.8552 - val_loss: 0.4241 - val_acc: 0.8463
Epoch 27/80
8121/8121 [==============================] - 8s 973us/step - loss: 0.3865 - acc: 0.8610 - val_loss: 0.4605 - val_acc: 0.8365
Epoch 28/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3975 - acc: 0.8527 - val_loss: 0.4401 - val_acc: 0.8419
Epoch 29/80
8121/8121 [==============================] - 8s 998us/step - loss: 0.3856 - acc: 0.8599 - val_loss: 0.4708 - val_acc: 0.8296
Epoch 30/80
8121/8121 [==============================] - 8s 982us/step - loss: 0.3959 - acc: 0.8567 - val_loss: 0.4599 - val_acc: 0.8433
Epoch 31/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3767 - acc: 0.8620 - val_loss: 0.4279 - val_acc: 0.8443
Epoch 32/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3820 - acc: 0.8637 - val_loss: 0.4287 - val_acc: 0.8473
Epoch 33/80
8121/8121 [==============================] - 8s

In [5]:
# for bat, lr in itertools.product(batch_S, learning_rate):
#     print(bat, lr)

In [6]:
import pprint
pprint.pprint(F1_result)

[array([0.78210692, 0.58941757, 0.90392228, 0.7522676 ]),
 array([0.79159909, 0.57669239, 0.90442679, 0.75448937]),
 array([0.76905081, 0.60450842, 0.90390831, 0.74750219]),
 array([0.76697001, 0.56622316, 0.90381395, 0.75523637]),
 array([0.80460213, 0.58552474, 0.90165807, 0.75286997]),
 array([0.7871668 , 0.56380356, 0.90283217, 0.74980707]),
 array([0.77751363, 0.5896029 , 0.90055801, 0.74005806]),
 array([0.76775888, 0.59426933, 0.90397249, 0.75388005]),
 array([0.76576212, 0.61504999, 0.90192391, 0.73725381]),
 array([0.77258374, 0.6046346 , 0.90345956, 0.74703007]),
 array([0.7989271 , 0.59520712, 0.90492798, 0.74554438])]


In [7]:
# array([0.79250755, 0.62614335, 0.90266635, 0.74891516])